# What is this notebook?
**OSM** or **O**pen**S**treet**M**ap is a free and open source online map of the world. By providing 4 [geodetic coordinates](https://en.wikipedia.org/wiki/Geodetic_coordinates) (left, right, top and bottom) surrounding the area of the map you want, this notebook can then download and output a **PNG** image file of that area.

In [1]:
from PIL import Image
import requests

import math
import io

### Conversion functions

In [2]:
# Formulas reference: https://en.wikipedia.org/wiki/Web_Mercator_projection

def lat2pix(r, z, floor=True):
    # computes Y-pixel number from latitude.
    # r: latitude in radians
    #    range should be: R < r <= R
    #    where R = 2*atan(exp(pi))-pi/2 = ~1.484422
    # z: zoom level, integer >= 0
    
    k = 2**(z+7) * (1 - math.log(math.tan(math.pi/4 + r/2))/math.pi)
    return math.floor(k) if floor else k


def pix2lat(y, z, round_=True):
    # computes latitude in radians from Y-pixel number.
    # y: Y-pixel number
    #    range should be: 0 <= y <= 2**(z+8)-1
    # z: zoom level, integer >= 0
    
    k = 2*math.atan(math.exp(math.pi*(1-y*2**-(z+7))))-math.pi/2
    return round(k, 8) if round_ else k


def lon2pix(r, z, floor=True):
    # computes X-pixel number from longitude.
    # r: longitude in radians
    #    range should be: -pi <= r < pi
    # z: zoom level, integer >= 0
    
    k = 2**(z+7)/math.pi * (r + math.pi)
    return math.floor(k) if floor else k


def pix2lon(x, z, round_=True):
    # computes longitude in radians from X-pixel number.
    # x: X-pixel number
    #    range should be: 0 <= x <= 2**(z+8)-1
    # z: zoom level, integer >= 0
    
    k = math.pi*(x*2**-(z+7)-1)
    return round(k, 8) if round_ else k

### Downloader and helper functions

In [3]:
def download_tiles(x1, x2, y1, y2, z, session=None):
    # downloads and yields bytes of tile images.
    # x1, x2, y1, y2: starting and ending (inclusive) tile numbers
    # z: zoom level
    
    session = session or requests.Session()
    for iy in range(y1, y2 + 1):
        for ix in range(x1, x2 + 1):
            url = f'https://tile.openstreetmap.org/{z}/{ix}/{iy}.png'
            response = session.get(url)
            response.raise_for_status()
            
            yield response.content


def concat_tiles(tiles, x, y):
    # concatenates tiles into a single image.
    # tiles: iterable of PIL.Image
    # x, y: number of tiles in the x and y axes
    
    im = Image.new('RGB', (256*x, 256*y))
    for i, tile in enumerate(tiles):
        ix, iy = i % x, i // x
        im.paste(tile, (256*ix, 256*iy))
    return im


def b2im(b):
    # reads image data in bytes and returns PIL.Image
    return Image.open(io.BytesIO(b))

### How to get the coordinates?

1. Go to https://www.openstreetmap.org
2. Move the map to where you want.
3. On the top left, click "Export".
4. On the left panel, click "Manually select a different area".
5. Move the box around to where you want.
6. On the left panel, Copy the 4 coordinates.
7. Use those for the next cell values.

In [4]:
# Zoom level is the detail of the map.
# The bigger the number the larger the PNG file.
zoom_level = 13

# Below variables are coordinate you've copied.
lat_top = 51.5153
lat_bottom = 51.4575
lon_left = -0.0810
lon_right =0.0810

# Output file name.
# Note: you may change file extension to get a different image format.
#       for example "map.jpg" will output a JPEG image.
file_name = 'map.png'

Run the following cell to initialize related variables and check for output image resolution.

In [5]:
pix_top = lat2pix(math.radians(lat_top), zoom_level)
pix_bottom = lat2pix(math.radians(lat_bottom), zoom_level)
pix_left = lon2pix(math.radians(lon_left), zoom_level)
pix_right = lon2pix(math.radians(lon_right), zoom_level)

width = pix_right - pix_left + 1
height = pix_bottom - pix_top + 1

x1, x2 = pix_left // 256, pix_right // 256
y1, y2 = pix_top // 256, pix_bottom // 256

x_offset = pix_left % 256
y_offset = pix_top % 256

print(f'Output image resolution will be: {width}x{height}')

Output image resolution will be: 944x542


If you satisfy with the resolution, run the next cell.

In [6]:
im = concat_tiles(map(b2im, download_tiles(x1, x2, y1, y2, zoom_level)), x2-x1+1, y2-y1+1)
im = im.crop((x_offset, y_offset, x_offset + width, y_offset + height))
im.save(file_name)